# Data Exploration - this is updated 11:28AM Tues

## 1. Importing data into DataFrames

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_mojo_gross = pd.read_csv('bom.movie_gross.csv.gz')
df_imdb_name_basics = pd.read_csv('imdb.name.basics.csv.gz')
df_imdb_title = pd.read_csv('imdb.title.akas.csv.gz')
df_imdb_basics = pd.read_csv('imdb.title.basics.csv.gz')
df_imdb_title_crew = pd.read_csv('imdb.title.crew.csv.gz')
df_imdb_title_principals = pd.read_csv('imdb.title.principals.csv.gz')
df_imdb_title_ratings = pd.read_csv('imdb.title.ratings.csv.gz')
df_rt_movie_info = pd.read_csv('rt.movie_info.tsv.gz', sep='\t', encoding = 'unicode_escape')
df_rt_reviews = pd.read_csv('rt.reviews.tsv.gz', sep='\t', encoding = 'unicode_escape')
df_tmdb_movies = pd.read_csv('tmdb.movies.csv.gz')
df_tn_movie_budgets = pd.read_csv('tn.movie_budgets.csv.gz')

In [ ]:
list_of_data = [df_mojo_gross, df_imdb_name_basics, df_imdb_title, df_imdb_basics, df_imdb_title_crew,
               df_imdb_title_principals, df_imdb_title_ratings, df_rt_movie_info, df_rt_reviews, df_tmdb_movies, 
                df_tn_movie_budgets]

## 2. Previewing DataFrames

In [ ]:
for df in list_of_data:
    display(df.head())

In [ ]:
for df in list_of_data:
    display(df.shape)

## 3. Visualizing DataFrames

In [ ]:
df_tn_movie_budgets.info()

Converting production_budget, domestic_gross and worldwide_gross from strings into floats:

In [ ]:
df_tn_movie_budgets['production_budget_edited'] = df_tn_movie_budgets.production_budget.str.replace('$', '')
df_tn_movie_budgets['production_budget_edited'] = df_tn_movie_budgets['production_budget_edited'].str.replace(',', '')
df_tn_movie_budgets['production_budget_edited'] = df_tn_movie_budgets['production_budget_edited'].astype(float)

df_tn_movie_budgets['domestic_gross_edited'] = df_tn_movie_budgets.domestic_gross.str.replace('$', '')
df_tn_movie_budgets['domestic_gross_edited'] = df_tn_movie_budgets['domestic_gross_edited'].str.replace(',', '')
df_tn_movie_budgets['domestic_gross_edited'] = df_tn_movie_budgets['domestic_gross_edited'].astype(float)

df_tn_movie_budgets['worldwide_gross_edited'] = df_tn_movie_budgets.worldwide_gross.str.replace('$', '')
df_tn_movie_budgets['worldwide_gross_edited'] = df_tn_movie_budgets['worldwide_gross_edited'].str.replace(',', '')
df_tn_movie_budgets['worldwide_gross_edited'] = df_tn_movie_budgets['worldwide_gross_edited'].astype(float)

df_tn_movie_budgets.head()

Making the basis in $M.

In [ ]:
df_tn_movie_budgets['production_budget_edited'] = df_tn_movie_budgets['production_budget_edited'] / 1000000
df_tn_movie_budgets['domestic_gross_edited'] = df_tn_movie_budgets['domestic_gross_edited'] / 1000000
df_tn_movie_budgets['worldwide_gross_edited'] = df_tn_movie_budgets['worldwide_gross_edited'] / 1000000

df_tn_movie_budgets.head()

In [ ]:
ax = sns.scatterplot(x=df_tn_movie_budgets['production_budget_edited'], y=df_tn_movie_budgets['worldwide_gross_edited'])

ax.set_title('Production Budget vs. Worldwide Gross ($M)')
ax.set_ylabel('Worldwide Gross ($M)')
ax.set_xlabel('Production Budget ($M)')

ax.text(df_tn_movie_budgets.production_budget_edited[df_tn_movie_budgets.movie=='Avatar'],
        df_tn_movie_budgets.worldwide_gross_edited[df_tn_movie_budgets.movie=='Avatar'],
         "Avatar", color='red');

Calculating an ROI figure.

In [ ]:
df_tn_movie_budgets['ROI'] = df_tn_movie_budgets['worldwide_gross_edited'] / df_tn_movie_budgets['production_budget_edited']

df_tn_movie_budgets.head()

In [ ]:
df_tn_movie_budgets_ROIaboveX = df_tn_movie_budgets[df_tn_movie_budgets['ROI'] >= 5]

df_tn_movie_budgets_ROIaboveX.head()

Showing movies with ROI > 5

In [ ]:
fig, ax = plt.subplots()

ax.scatter(x=df_tn_movie_budgets['production_budget_edited'], y=df_tn_movie_budgets['worldwide_gross_edited'])
ax.scatter(x=df_tn_movie_budgets_ROIaboveX['production_budget_edited'], 
           y=df_tn_movie_budgets_ROIaboveX['worldwide_gross_edited'], label='ROI Above 5')

ax.set_title('Production Budget vs. Worldwide Gross ($M)')
ax.set_ylabel('Worldwide Gross ($M)')
ax.set_xlabel('Production Budget ($M)')
ax.legend(loc='upper left');

In [ ]:
df_tn_movie_budgets_ROIaboveX.sort_values('ROI', ascending=False)

Only interested in relevant movies - post 2009

In [ ]:
df_tn_movie_budgets_ROIaboveX['Year'] = df_tn_movie_budgets_ROIaboveX['release_date'].str[-4:]*1
df_tn_movie_budgets_ROIaboveX['Year'] = df_tn_movie_budgets_ROIaboveX['Year'].astype(int)
df_tn_movie_budgets_ROIaboveX.head()

In [ ]:
df_tn_movie_budgets_post2009 = df_tn_movie_budgets_ROIaboveX[df_tn_movie_budgets_ROIaboveX['Year'] >= 2009]
df_tn_movie_budgets_post2009.head()

In [ ]:
df_tn_movie_budgets_post2009.shape

In order to translte the genre_ids into meaningful genre names, we must retrieve the genre key from TMDB:

In [ ]:
api_key = "d012e23fce658ca5d5707ca650488781"

import requests

url = 'https://api.themoviedb.org/3/genre/movie/list?api_key={}&language=en-US'.format(api_key)

tmdb_genre = requests.get(url)
print(tmdb_genre)
print(type(tmdb_genre.content))
print(tmdb_genre.text)

In [ ]:
tmdb_genre = tmdb_genre.json()
tmdb_genre

Retrieving genre keys from the TMDB api and converting into a python dictionary to be used later to convert genre_id to meaningful labels

In [ ]:
tmdb_genre_dict = {}

for x in range(len(tmdb_genre['genres'])):
    entry = {tmdb_genre['genres'][x]['id']: tmdb_genre['genres'][x]['name']}
    tmdb_genre_dict.update(entry)

tmdb_genre_dict

Converting the dictionary keys from int to str to match genre_id

In [ ]:
keys_values = tmdb_genre_dict.items()

tmdb_genre_dict = {str(key): str(value) for key, value in keys_values}
tmdb_genre_dict

In [ ]:
df_tmdb_movies.head()

In [ ]:
df_tmdb_movies['genre_ids'][0]

Cleaning genre_id column to parse through in the future 

In [ ]:
df_tmdb_movies['genre'] = df_tmdb_movies['genre_ids'].str.strip('[]')
df_tmdb_movies['genre'] = df_tmdb_movies['genre'].str.replace(' ', '')
df_tmdb_movies['genre'] = df_tmdb_movies['genre'].str.split(',')

df_tmdb_movies.head()

In [ ]:
# display(df_tn_movie_budgets)
# display(df_tmdb_movies)

# df_tn_movie_budgets.query("movie == 'Harry Potter and the Deathly Hallows: Part 1'")
print('TMDB DF SHAPE:',df_tmdb_movies.shape)
print('TNBUD DF SHAPE:',df_tn_movie_budgets.shape)

We had to decide how we want to join our two tables. Our questions are mainly centered around budget so we came to the conclusion that we will only factor movies that we have budget and rating data for. 

In [ ]:
df_tn_movie_budgets['release_date'] = pd.to_datetime(df_tn_movie_budgets['release_date'])
df_tmdb_movies['release_date'] = pd.to_datetime(df_tmdb_movies['release_date'])
tn_tmdb_merged_df = df_tmdb_movies.merge(df_tn_movie_budgets, how='inner', left_on=['original_title','release_date'], 
                                         right_on=['movie','release_date'])
tn_tmdb_merged_df

First, we expanded every element in the list of the cleaned `genre_id`. Then mapped each value based on their keys from the TMDB API Dictionary.

In [ ]:
def genreID_coverter(string):
    if string is not None and len(string) > 0:
        return tmdb_genre_dict[string]

In [ ]:
df_tmdb_movies_genresplit = pd.DataFrame(tn_tmdb_merged_df['genre'].values.tolist())

for i in range (6):
    df_tmdb_movies_genresplit[i] = df_tmdb_movies_genresplit[i].apply(genreID_coverter)

df_tmdb_movies_genresplit.columns = ['genre1', 'genre2','genre3','genre4','genre5','genre6', 'genre7']

df_tmdb_movies_genresplit

In [ ]:
df_MAIN = pd.concat([tn_tmdb_merged_df, df_tmdb_movies_genresplit], axis=1)
df_MAIN.drop(columns=['Unnamed: 0', 'id_x', 'genre_ids', 'domestic_gross', 'worldwide_gross','production_budget', 'id_y', 
                     'original_title', 'movie'], inplace=True)


def queryGenre(genreTypeString):
    dataFrameToBeReturned = df_MAIN.query("""
                (genre1 == '{}') or \
                (genre2 == '{}') or \
                (genre3 == '{}') or \
                (genre4 == '{}') or \
                (genre5 == '{}') or \
                (genre6 == '{}') or \
                (genre7 == '{}')
                """.format(genreTypeString,genreTypeString,genreTypeString,genreTypeString,
                           genreTypeString,genreTypeString,genreTypeString))
    return dataFrameToBeReturned

genre_df_dictionary = {}
for key, value in tmdb_genre_dict.items():
    genre_df_dictionary[value] = (queryGenre(value))

genre_df_dictionary['Western']

In [ ]:
df_MAIN.corr()

In [ ]:
df_genre_counts = pd.DataFrame(columns=['genre', 'genre_counts'])

new_dict = {}
for key, value in genre_df_dictionary.items():
    new_dict[key] = len(genre_df_dictionary[key])

value_df_genre_counts = df_genre_counts.append(new_dict, ignore_index=True).T.iloc[2:]

value_df_genre_counts

In [ ]:
value_df_genre_counts.sort_values(by=0, ascending=False)

In [ ]:
type(genre_df_dictionary['Action']['production_budget_edited'][1])

In [ ]:
x_slope1 = np.linspace(0,4000)
y_slope1 = np.linspace(0,4000)

col_nums = 4
row_nums = 5
plt.figure(figsize=(20, 20))
for i, (key, v) in enumerate(genre_df_dictionary.items(), 1):
    plt.subplot(row_nums, col_nums, i)
    try:
        p = sns.scatterplot(data=v, x='production_budget_edited', y='worldwide_gross_edited').set(
            xlim=(genre_df_dictionary[key]['production_budget_edited'].min(), genre_df_dictionary[key]['production_budget_edited'].max()), 
            ylim=(genre_df_dictionary[key]['worldwide_gross_edited'].min(), genre_df_dictionary[key]['worldwide_gross_edited'].max()))
        plt.plot(x_slope1, y_slope1, color='r')
        plt.title(key)
    except:
        continue
plt.tight_layout()

In [ ]:
# for key in genre_df_dictionary.keys():
#     print(key, genre_df_dictionary[key].corr())
   

genre_df_dictionary['Action'].corr()


    
# corr_dict_by_genre = {}
# for key, value in genre_df_dictionary.items():
#     corr_dict_by_genre[key] = genre_df_dictionary[key].corr

In [ ]:
season_code_arr = [month%12 // 3 + 1 for month in range(1, 13)]
season_code_to_month = {1: 'Winter',
                         2: 'Spring',
                         3: 'Summer',
                         4: 'Autumn'}


df_season_test = df_MAIN

df_season_test['season'] = pd.DatetimeIndex(df_season_test['release_date']).month
df_season_test

In [ ]:
df_season_test['season'] = df_season_test['season'] %12 // 3 + 1
df_season_test

In [ ]:
df_season_test['season'] = df_season_test.replace({'season': season_code_to_month})

df_season_test.head()

In [ ]:
list_of_genres = ['Drama', 'Comedy', 'Thriller', 'Action', 'Adventure', 'Crime', 'Science Fiction', 'Romance', 'Horror',
                 'Family', 'Fantasy', 'Mystery']

col_nums = 3
row_nums = 4
plt.figure(figsize=(20, 20))
for key in list_of_genres:
    p = sns.scatterplot(data=genre_df_dictionary[key], x='production_budget_edited', y='worldwide_gross_edited')
    plt.title(key)

plt.tight_layout()